In [ ]:
import os
from tqdm import tqdm
import pandas as pd
from Bio import Entrez
Entrez.email = "example@example.com"
from urllib.error import HTTPError

In [ ]:
def get_pmids_from_one_day(topic, date):
    handle = Entrez.esearch(db="pubmed", 
                            term= topic + "[MeSH Terms]", 
                            retmax=1000,
                            datetype="pdat", 
                            mindate=date, 
                            maxdate=date, )
    record = Entrez.read(handle)
    ids = record["IdList"]
    return ids

def get_pmids_from_period(topic, span):
    handle = Entrez.esearch(db="pubmed", 
                            term= topic + "[MeSH Terms]", 
                            retmax=1000,
                            datetype="pdat", 
                            mindate=span[0], 
                            maxdate=span[1], )
    record = Entrez.read(handle)
    ids = record["IdList"]
    return ids


def get_xml(id):
    handle = Entrez.efetch(db="pubmed", 
                            id=id, 
                            retmode="xml", 
                            rettype="abstract")
    xml = handle.read()
    # pp = pprint.PrettyPrinter(indent=4)
    # pp.pprint(json.dumps(xmltodict.parse(xml)))
    return xml


def get_xmls_ids(ids):
    pairs = []
    for id in tqdm(ids):
        xml = get_xml(id)
        pairs += [(id, xml)]
    return pairs

import csv
def save_span(span, list_of_pairs):
    file_name = span[0].replace("/", "_") + "__" + span[1].replace("/", "_") + ".csv"
    with open(file_name,'w') as out:
        csv_out=csv.writer(out, dialect='unix')
        # csv_out.writerow(['name','num'])
        # for row in list_of_pairs:
        #     csv_out.writerow(row)
        csv_out.writerows(list_of_pairs)
    return file_name

def save_date(date, list_of_pairs):
    file_name = date.replace("/", "_") + ".csv"
    with open(file_name,'w') as out:
        csv_out=csv.writer(out, dialect='unix')
        # csv_out.writerow(['name','num'])
        # for row in list_of_pairs:
        #     csv_out.writerow(row)
        csv_out.writerows(list_of_pairs)
        print(f"Saved {len(list_of_pairs)} papers published on {date}.")

In [ ]:
month = '03'

for day in range(1, 32):
    date = "2022/" + month + "/" + f'{day:02}'
    print(f"Starting with {date}")
    ids = get_pmids_from_one_day("breast cancer", date)
    if len(ids) == 0:
        break
    print(f"Number of papers published on  {date}': {len(ids)}")
    pairs = get_xmls_ids(ids)
    file_name = save_date(date, pairs)

In [ ]:
# file_name = '2022_01_02__2022_01_10.csv'
# df = pd.read_csv(file_name, header=None)
# df